In [6]:
import Pkg; Pkg.activate(".")

  Activating project at `~/SAFT_ML`


In [7]:
using CUDA
using Flux
using Flux: onehotbatch, onecold
using Flux.Losses: logitcrossentropy
using Flux.Data: DataLoader
using GeometricFlux
using GeometricFlux.Datasets
using GraphSignals
using Graphs
using Parameters: @with_kw
using ProgressMeter: Progress, next!
using Statistics
using Random

In [30]:
function load_data(dataset, batch_size, train_repeats=32, test_repeats=2)
    s, t = dataset[1].edge_index
    g = Graphs.Graph(dataset[1].num_nodes)
    for (i, j) in zip(s, t)
        Graphs.add_edge!(g, i, j)
    end

    data = dataset[1].node_data
    X, y = data.features, onehotbatch(data.targets, 1:7)
    # return dataset
    # return X, y
    train_idx, test_idx = data.train_mask, data.val_mask

    # (train_X, train_y) dim: (num_features, target_dim) × 2708 × train_repeats
    train_X, train_y = repeat(X, outer=(1,1,train_repeats)), repeat(y, outer=(1,1,train_repeats))
    # (test_X, test_y) dim: (num_features, target_dim) × 2708 × test_repeats
    test_X, test_y = repeat(X, outer=(1,1,test_repeats)), repeat(y, outer=(1,1,test_repeats))

    add_all_self_loops!(g)
    fg = FeaturedGraph(g)
    train_loader = DataLoader((train_X, train_y), batchsize=batch_size, shuffle=true)
    test_loader = DataLoader((test_X, test_y), batchsize=batch_size, shuffle=true)
    # return train_loader, test_loader, fg, train_idx, test_idx
    return test_X, test_y, fg
end

function add_all_self_loops!(g)
    for i in vertices(g)
        add_edge!(g, i, i)
    end
    return g
end

@with_kw mutable struct Args
    η = 0.01                # learning rate
    batch_size = 8          # batch size
    epochs = 20             # number of epochs
    seed = 0                # random seed
    cuda = false            # use GPU
    heads = 2               # attention heads
    input_dim = 1433        # input dimension
    hidden_dim = 16         # hidden dimension
    target_dim = 7          # target dimension
    dataset = Cora          # dataset to train on
end

# Cora dataset has:
# - 2708 nodes
# - 1433 features
# - 7 classes
# - 10556 edges

# Nodes represent documents, edges represent citations.

## Loss: cross entropy
model_loss(model, X, y, idx) =
    logitcrossentropy(model(X)[:,idx,:], y[:,idx,:])

accuracy(model, X::AbstractArray, y::AbstractArray, idx) =
    mean(onecold(softmax(cpu(model(X))[:,idx,:])) .== onecold(cpu(y)[:,idx,:]))

accuracy(model, loader::DataLoader, device, idx) =
    mean(accuracy(model, X |> device, y |> device, idx) for (X, y) in loader)

accuracy (generic function with 2 methods)

In [32]:
args = Args()
# train_loader, test_loader, fg, train_idx, test_idx = load_data(args.dataset(), args.batch_size)
test_X, test_y, fg = load_data(args.dataset(), args.batch_size)
# args = Args()
# args.seed > 0 && Random.seed!(args.seed)
# X, y = load_data(args.dataset(), args.batch_size)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0;;; 0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], FeaturedGraph:
	Undirected graph with (#V=2708, #E=7986) in adjacency matrix)

In [3]:

function train(; kws...)
    # load hyperparamters
    args = Args(; kws...)
    args.seed > 0 && Random.seed!(args.seed)

    # GPU config
    if args.cuda && CUDA.has_cuda()
        device = gpu
        CUDA.allowscalar(false)
        @info "Training on GPU"
    else
        device = cpu
        @info "Training on CPU"
    end

    # load Cora from Planetoid dataset
    train_loader, test_loader, fg, train_idx, test_idx = load_data(args.dataset(), args.batch_size)

    @info "Data loaded, building model..."

    # build model
    model = Chain(
        WithGraph(fg, GATConv(args.input_dim => args.hidden_dim, heads=args.heads)),
        Dropout(0.6),
        WithGraph(fg, GATConv(args.hidden_dim * args.heads => args.target_dim, heads=args.heads, concat=false)),
    ) |> device

    @info "Model built, loading optimiser and parameters..."

    # Adam optimizer
    opt = Adam(args.η)

    # parameters
    ps = Flux.params(model)

    # training
    train_steps = 0
    @info "Starting Training, total $(args.epochs) epochs"
    for epoch = 1:args.epochs
        @info "Epoch $(epoch)"
        progress = Progress(length(train_loader))

        for (X, y) in train_loader
            X, y, device_idx = X |> device, y |> device, train_idx |> device
            loss, back = Flux.pullback(() -> model_loss(model, X, y, device_idx), ps)
            train_acc = accuracy(model, train_loader, device, train_idx)
            test_acc = accuracy(model, test_loader, device, test_idx)
            grad = back(1.0f0)
            Flux.Optimise.update!(opt, ps, grad)

            # progress meter
            next!(progress; showvalues=[
                (:loss, loss),
                (:train_accuracy, train_acc),
                (:test_accuracy, test_acc)
            ])

            train_steps += 1
        end
    end

    return model, args
end

model, args = train()

┌ Info: Training on CPU
└ @ Main /home/luc/SAFT_ML/3_gnn_example.ipynb:66
┌ Info: Data loaded, building model...
└ @ Main /home/luc/SAFT_ML/3_gnn_example.ipynb:72
